In [ ]:
# using Pkg
# Pkg.add("LinearAlgebra")
# Pkg.add("Statistics")
# Pkg.add("LsqFit")
# Pkg.add("CSV")
# Pkg.add("DataFrames")
# Pkg.add("IterTools")
# Pkg.add("DataStructures")
# Pkg.add("JuMP")
# Pkg.add("Ipopt")
# Pkg.add("ForwardDiff")
# Pkg.add("OrderedCollections")
# Pkg.add("Dates")
# Pkg.add("Plots")
# Pkg.add("StatsPlots")
# Pkg.add("Distributions")
# Pkg.add("Random")
# Pkg.add("Optim")
# Pkg.add("Printf")
# Pkg.add("StatsBase")
# Pkg.add("LaTeXStrings")

In [ ]:
using LinearAlgebra
using Statistics
using LsqFit
using CSV
using DataFrames
using Base.Iterators
using DataStructures
using JuMP
using Ipopt
using ForwardDiff
using OrderedCollections
using Dates
using Plots
using StatsPlots
using Distributions
using Random
using StatsBase
using LaTeXStrings 

In [ ]:
# --- 1. CIR Model Parameters ---
const TRUE_KAPPA = 0.6 # Mean reversion speed
const TRUE_THETA = 0.135 # Long-term mean
const TRUE_SIGMA = 0.09  # Volatility

const N_SIM = 100000 # Simulations

In [ ]:
TRUE_SIGMA^(2)

In [ ]:
2*TRUE_KAPPA*TRUE_THETA - TRUE_SIGMA^2

In [ ]:
"""
    simulate_cir_paths(n_sim; ...)

Simulates interest rate paths using the Cox-Ingersoll-Ross (CIR) model.
(Função da resposta anterior, modificada para não plotar por padrão)
"""
function simulate_cir_paths(n_sim::Int; kappa=TRUE_KAPPA, theta=TRUE_THETA, sigma=TRUE_SIGMA)
    # Simulation Parameters
    T = 1.0      # Time horizon in years
    N = 22        # Number of time steps
    dt = T / N     # Time step size
    r0 = 0.13

    rates = zeros(N + 1, n_sim)
    rates[1, :] .= r0
    Z = rand(Normal(0, 1), N, n_sim)

    for t in 1:N
        sqrt_r_t = @. sqrt(max(0, rates[t, :]))
        rates[t+1, :] = @. rates[t, :] + kappa * (theta - rates[t, :]) * dt +
                         sigma * sqrt_r_t * sqrt(dt) * Z[t, :]
    end
    return rates, T, N, dt, r0
end

In [ ]:
# --- Bloco Principal de Execução e Plotagem ---

# 2. Gerar os dados da simulação chamando sua função
paths, T, N, dt, r0 = simulate_cir_paths(N_SIM)

# 3. Preparar os dados para o gráfico
# Criar o vetor de tempo para o eixo X
t_points = range(0, stop=T, length=N + 1)

# Selecionar os primeiros 10 caminhos para uma visualização limpa
paths_to_plot = paths[:, 1:10]

# 4. Criar e exibir o gráfico
p = plot(t_points, paths_to_plot,
         legend = :topright, # Habilita a legenda para a linha de referência
         label = "",         # Remove o label dos 10 caminhos individuais
         title = "Simulação de 10 Caminhos da Taxa de Juros (Modelo CIR)",
         xlabel = "Tempo (Anos)",
         ylabel = "Taxa de Juros",
         grid = true)

# Adicionar uma linha horizontal para a média de longo prazo (theta)
hline!([TRUE_THETA],
       linestyle = :dash,
       color = :red,
       linewidth = 2,
       label = "Média de Longo Prazo (θ)")

# Exibir o plot final
display(p)

### Step 1: Find the SDE for the Transformed Process $Y_t$

The goal is to prove that for the estimator $Z_t = \frac{Y_t^2}{F(t) - F(0)}$, its expectation is $\mathbb{E}[Z_t] = \sigma^2$. This is equivalent to proving that $\mathbb{E}[Y_t^2] = \sigma^2(F(t) - F(0))$.

We begin with the Cox-Ingersoll-Ross (CIR) process $R_t$ and its expected path $\bar{r}_t$:
\begin{align*}
    dR_t &= \kappa(\theta - R_t)dt + \sigma\sqrt{R_t}dW_t \\
    \bar{r}_t &= \theta + (r_0 - \theta)e^{-\kappa t}
\end{align*}

And we have the $Y_t$ process:

$$ Y_t = e^{\kappa t}(R_t - \bar{r}_t) $$
Applying the product rule for Itô processes, the dynamics for $Y_t$ are:
\begin{align*}
    dY_t &= \kappa e^{\kappa t}(R_t - \bar{r}_t)dt + e^{\kappa t}d(R_t - \bar{r}_t) \\
         &= \kappa e^{\kappa t}(R_t - \bar{r}_t)dt + e^{\kappa t}\left(-\kappa(R_t - \bar{r}_t)dt + \sigma\sqrt{R_t}dW_t\right) \\
         &= \left( \kappa e^{\kappa t}(R_t - \bar{r}_t) - \kappa e^{\kappa t}(R_t - \bar{r}_t) \right)dt + e^{\kappa t}\sigma\sqrt{R_t}dW_t
\end{align*}
The drift terms cancel, leaving a driftless martingale starting at $Y_0 = 0$:
$$ dY_t = \sigma e^{\kappa t} \sqrt{R_t} dW_t $$

### section Step 2: Apply Itô's Lemma to $Y_t^2$

To find the dynamics for $g(Y_t) = Y_t^2$, we use Itô's Lemma:
$$ d(Y_t^2) = \frac{\partial g}{\partial Y} dY_t + \frac{1}{2} \frac{\partial^2 g}{\partial Y^2} d[Y, Y]_t $$
The components are:

- Derivatives: $\frac{\partial g}{\partial Y} = 2Y_t$ and $\frac{\partial^2 g}{\partial Y^2} = 2$.
- Quadratic Variation: $d[Y, Y]_t = (dY_t)^2 = (\sigma e^{\kappa t} \sqrt{R_t} dW_t)^2 = \sigma^2 e^{2\kappa t} R_t dt$.

Substituting these back into the lemma gives the SDE for $Y_t^2$:
$$ d(Y_t^2) = 2Y_t dY_t + \sigma^2 e^{2\kappa t} R_t dt $$

### Step 3: Integrate and Take the Expectation

We integrate the SDE for $Y_t^2$ from $0$ to $t$:
$$ Y_t^2 - Y_0^2 = \int_0^t 2Y_u dY_u + \int_0^t \sigma^2 e^{2\kappa u} R_u du $$
Since $Y_0 = e^0(R_0 - \bar{r}_0) = 1(r_0 - r_0) = 0$, this simplifies to:
$$ Y_t^2 = \int_0^t 2Y_u dY_u + \int_0^t \sigma^2 e^{2\kappa u} R_u du $$
Taking the expectation, $\mathbb{E}[\cdot]$, of both sides:
$$ \mathbb{E}[Y_t^2] = \mathbb{E}\left[\int_0^t 2Y_u dY_u\right] + \mathbb{E}\left[\int_0^t \sigma^2 e^{2\kappa u} R_u du\right] $$
A fundamental property of Itô integrals is that their expectation is zero. Thus, the first term vanishes:
$$ \mathbb{E}\left[\int_0^t 2Y_u dY_u\right] = 0 $$
For the second term, we can swap the expectation and integral by Fubini's theorem. Since $\sigma^2$ and $e^{2\kappa u}$ are deterministic, we have:
\begin{align*}
    \mathbb{E}[Y_t^2] &= \int_0^t \mathbb{E}[\sigma^2 e^{2\kappa u} R_u] du \\
                 &= \sigma^2 \int_0^t e^{2\kappa u} \mathbb{E}[R_u] du
\end{align*}
By definition, $\mathbb{E}[R_u] = \bar{r}_u$. Therefore:
$$ \mathbb{E}[Y_t^2] = \sigma^2 \int_0^t e^{2\kappa u} \bar{r}_u du $$

### Step 4: Conclude the Proof

We define the function $F(t)$ such that its increment over $[0, t]$ is the integral we just found:
$$ F(t) - F(0) \equiv \int_0^t e^{2\kappa u} \bar{r}_u du $$
By substituting this definition into our result from Step 3, we arrive at the desired identity:
$$ \mathbb{E}[Y_t^2] = \sigma^2 (F(t) - F(0)) $$
Finally, for the estimator $Z_t = \frac{Y_t^2}{F(t) - F(0)}$, its expectation is:
$$ \mathbb{E}[Z_t] = \frac{\mathbb{E}[Y_t^2]}{F(t) - F(0)} = \frac{\sigma^2 (F(t) - F(0))}{F(t) - F(0)} = \sigma^2 \quad \blacksquare $$


In [ ]:
"""
calculate_z_trajectories(paths, T, N, dt, r0, kappa, theta)

Calculates the Z(t) trajectory for an entire matrix of paths using vectorization.
"""
function calculate_z_trajectories(paths::Matrix{Float64}, T::Float64, N::Int, dt::Float64, r0::Float64, kappa::Float64, theta::Float64)
    num_points, num_paths = size(paths)
    t_points = 0:dt:T
    
    # 1. Calculate auxiliary vectors and matrices
    # r_bar and F are vectors, as they only depend on time
    r_bar = @. theta + (r0 - theta) * exp(-kappa * t_points)
    
    # y_matrix is a matrix, calculated for all paths at once via broadcasting
    y_matrix = @. exp(kappa * t_points) * (paths - r_bar)

    F = @. exp(kappa*t_points)/kappa * (r0 + theta*(exp(kappa*t_points)/2 - 1))
    
    # 2. Vectorized calculation of Z(t) trajectories
    # Numerators: Square every element in the y_matrix
    numerators_matrix = y_matrix.^2
    
    # Denominators: Create a vector of denominators, F(t) - F(0)
    denominators_vector = F .- F[1]
    
    # Divide each row of the numerator matrix by the corresponding element
    # in the denominators_vector. Broadcasting handles this automatically.
    z_trajectories = numerators_matrix ./ denominators_vector
    
    # The value at t=0 is undefined (division by zero), so set the first row to NaN
    z_trajectories[1, :] .= NaN
    
    return z_trajectories
end

In [ ]:
# --- Main Execution Block ---

# I will use the same paths, T, N, dt, r0, TRUE_KAPPA, TRUE_THETA variables that was defined above to simulate CIR paths

z_trajectories_matrix = calculate_z_trajectories(paths, T, N, dt, r0, TRUE_KAPPA, TRUE_THETA)
println("Z(t) calculation complete.")

# 1. Calculate the variance at each point in time
# Iterate over each row (each point in time) of the Z trajectories matrix
# and calculate the variance across all simulations.
mean_over_time = [mean(filter(isfinite, row)) for row in eachrow(z_trajectories_matrix)]

# 2. Prepare for plotting
# The time axis for the plot
t_points = 0:dt:T

# Plot the variance over time
p = plot(t_points, mean_over_time,
         label = "Mean of Z(t) estimated",
         title = "Evolution of the Z(t) Estimator's Mean",
         xlabel = "Time",
         ylabel = "E[Z(t)]",
         legend = :topleft,
         linewidth = 2)

display(p)

#### Variance converges to:

$$ Var(Z_t) \approx 3\sigma^4 \left( 1+ \frac{\sigma^2}{\theta\kappa} \right)$$

In [ ]:
println(3*TRUE_SIGMA^(4)*(1+TRUE_SIGMA^(2)/(TRUE_THETA*TRUE_KAPPA) ))

In [ ]:
# --- Main Execution Block ---

# I will use the same paths, T, N, dt, r0, TRUE_KAPPA, TRUE_THETA variables that was defined above to simulate CIR paths

z_trajectories_matrix = calculate_z_trajectories(paths, T, N, dt, r0, TRUE_KAPPA, TRUE_THETA)
println("Z(t) calculation complete.")

# 1. Calculate the variance at each point in time
# Iterate over each row (each point in time) of the Z trajectories matrix
# and calculate the variance across all simulations.
variance_over_time = [var(filter(isfinite, row)) for row in eachrow(z_trajectories_matrix)]

# 2. Prepare for plotting
# The time axis for the plot
t_points = 0:dt:T

# Plot the variance over time
p = plot(t_points, variance_over_time,
         label = "Variance of Z(t) estimated",
         title = "Evolution of the Z(t) Estimator's Variance",
         xlabel = "Time",
         ylabel = "Var(Z(t))",
         legend = :topleft,
         linewidth = 2)

display(p)